## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-21-06-54-13 +0000,nypost,Chilling details emerge in killing of suicidal...,https://nypost.com/2025/11/21/us-news/uk-woman...
1,2025-11-21-06-52-58 +0000,bbc,US and Asia stocks slide as AI jitters persist,https://www.bbc.com/news/articles/cwy1yl1202yo...
2,2025-11-21-06-47-15 +0000,bbc,Community food boxes 'selling out within minutes',https://www.bbc.com/news/articles/cr7mdn89lemo...
3,2025-11-21-06-43-00 +0000,wsj,U.S. Treasurys Seen as Outperformers Next Year,https://www.wsj.com/articles/u-s-treasurys-see...
4,2025-11-21-06-39-00 +0000,wsj,Japan Approves $135 Billion Stimulus Shot to H...,https://www.wsj.com/economy/japan-approves-135...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
109,trump,44
301,new,19
96,will,14
98,california,12
71,ukraine,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
160,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...,107
206,2025-11-20-20-12-26 +0000,latimes,Trump administration plans to open Pacific Coa...,https://www.latimes.com/environment/story/2025...,87
196,2025-11-20-20-38-00 +0000,nypost,Judge orders Trump to end National Guard deplo...,https://nypost.com/2025/11/20/us-news/judge-or...,85
154,2025-11-20-22-26-31 +0000,bbc,Trump signs bill ordering US justice departmen...,https://www.bbc.com/news/articles/cly2y40lj5go...,83
223,2025-11-20-19-24-00 +0000,wsj,President Trump called for the arrest of Democ...,https://www.wsj.com/politics/policy/trump-call...,82


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
160,107,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...
64,61,2025-11-21-01-36-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...
260,58,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...
392,51,2025-11-20-08-35-04 +0000,wsj,"Stock Market News, Nov. 20, 2025: Stock-Market...",https://www.wsj.com/livecoverage/jobs-report-b...
261,50,2025-11-20-17-20-17 +0000,nypost,Rubio casts doubt on pro-Russian Ukraine peac...,https://nypost.com/2025/11/20/world-news/rubio...
145,47,2025-11-20-22-30-24 +0000,bbc,When will the Epstein files be released and wi...,https://www.bbc.com/news/articles/cq50nvq8znvo...
108,42,2025-11-20-23-52-41 +0000,cbc,The Fugees' Pras Michel handed 14-year sentenc...,https://www.cbc.ca/news/world/pras-michel-fuge...
104,41,2025-11-20-23-59-45 +0000,nypost,Mayor Eric Adams’ administration comes out swi...,https://nypost.com/2025/11/20/us-news/mayor-er...
250,38,2025-11-20-17-55-23 +0000,nypost,Israeli strikes kill 27 in Gaza after alleged ...,https://nypost.com/2025/11/20/world-news/israe...
179,38,2025-11-20-21-33-00 +0000,wsj,The Energy Department is eliminating two of it...,https://www.wsj.com/politics/policy/energy-dep...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
